In [ ]:
import openai
import os

openai.api_key = os.getenv("PULZE_API_KEY")
openai.api_base = "https://api.pulze.ai/v1"

text_response = openai.Completion.create(
  model="pulze-v0",
  prompt="Say Hello World!",
)

chat_response = openai.ChatCompletion.create(
  model="pulze-v0",
  messages=[{
    "role": "user",
    "content": "Say Hello World!"
  }],
)

print(text_response, chat_response)

In [7]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = 'https://api.openai.com/v1' # default
embedding_model_name = "text-embedding-ada-002"
embedding_model = OpenAIEmbeddings(model=embedding_model_name)
read_vector_store = Chroma(
        persist_directory="langchain-chroma-pulze-docs", embedding_function=embedding_model
    )
query_results = read_vector_store.similarity_search("How do I use Pulze?")
query_results

[Document(page_content='Pulze.ai\n home page\nSearch...\nTry for free\nTry for free\nSearch\nNavigation\nDocumentation\nTry for free\nOVERVIEW\nIntroductions\nGetting Started\nModels\nFEATURES\nLLMOps\nKey Features\nCustom headers\nLabeling\nMonitoring\nLogging\nRating\nPlayground\nTUTORIALS\nExample app\nAPI REFERENCE\nPrompts\nLogs\nApps\nOVERVIEW\nGetting Started\n\u200b\nWelcome to Pulze.ai\n\n\nPulze.ai is a powerful language model router trained to be an effective tool for various Natural Language Processing (NLP) tasks. Just like OpenAI’s GPT-4 and Anthropic’s Claude, our product is designed for seamless integration with your existing applications, making it more accessible for developers and creative technologists.\n\n\n\u200b\nWays to interact with Pulze.ai:', metadata={'source': 'https://docs.pulze.ai/overview/getting-started', 'title': 'Getting Started'}),
 Document(page_content='Pulze.ai\n home page\nSearch...\nTry for free\nTry for free\nSearch\nNavigation\nDocumentation\n

In [8]:
print(query_results[0].page_content)

Pulze.ai
 home page
Search...
Try for free
Try for free
Search
Navigation
Documentation
Try for free
OVERVIEW
Introductions
Getting Started
Models
FEATURES
LLMOps
Key Features
Custom headers
Labeling
Monitoring
Logging
Rating
Playground
TUTORIALS
Example app
API REFERENCE
Prompts
Logs
Apps
OVERVIEW
Getting Started
​
Welcome to Pulze.ai


Pulze.ai is a powerful language model router trained to be an effective tool for various Natural Language Processing (NLP) tasks. Just like OpenAI’s GPT-4 and Anthropic’s Claude, our product is designed for seamless integration with your existing applications, making it more accessible for developers and creative technologists.


​
Ways to interact with Pulze.ai:


In [9]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

# LLM
llm = ChatOpenAI()

# Prompt 
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How can I assist you today?')],
 'text': 'Hello! How can I assist you today?'}

In [10]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    read_vector_store.as_retriever(), 
    "search_pulze_docs",
    "Searches and returns documents regarding Pulze."
)
tools = [tool]

In [11]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature = 0)

agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

In [12]:
result = agent_executor({"input": "How do I use Pulze with LLM Ops best practices in mind?"})
result["output"]



> Entering new AgentExecutor chain...

Invoking: `search_pulze_docs` with `Pulze LLM Ops best practices`


[Document(page_content='Pulze.ai\n home page\nSearch...\nTry for free\nTry for free\nSearch\nNavigation\nDocumentation\nTry for free\nOVERVIEW\nIntroductions\nGetting Started\nModels\nFEATURES\nLLMOps\nKey Features\nCustom headers\nLabeling\nMonitoring\nLogging\nRating\nPlayground\nTUTORIALS\nExample app\nAPI REFERENCE\nPrompts\nLogs\nApps\nFEATURES\nLLMOps\n\u200b\nPulze.ai: The Ultimate Solution for Large Language Model Operations (LLMOps)\n\n\n\u200b\nIntroduction\n\n\nAs software development evolved over the years, we have seen a transformation from DevOps to MLOps and now the new era of Large Language Model Operations (LLMOps).', metadata={'source': 'https://docs.pulze.ai/features/LLMOps', 'title': 'LLMOps'}), Document(page_content='DevOps emerged as a methodology for ensuring better collaboration and productivity between development and operations teams. The focus of MLOps

'To use Pulze with LLM Ops best practices in mind, you can leverage the following key features of Pulze.ai:\n\n1. App Hosting and Orchestration: Pulze.ai provides a platform for hosting and orchestrating your applications that utilize large language models. It allows you to deploy and manage your applications efficiently.\n\n2. Validation and Benchmarking: Pulze.ai offers validation and benchmarking capabilities to ensure the performance and accuracy of your large language models. You can validate your models against various metrics and benchmarks to ensure their quality.\n\n3. LLM Caching: Pulze.ai implements advanced LLM caching techniques to optimize model performance and reduce latency. This is particularly important for applications that require real-time or near real-time responses.\n\n4. LLM Logging and Observability: Pulze.ai provides comprehensive logging and observability features. You can monitor all model requests and performance metrics, gain insights into model performanc

In [13]:
print(result["output"])

To use Pulze with LLM Ops best practices in mind, you can leverage the following key features of Pulze.ai:

1. App Hosting and Orchestration: Pulze.ai provides a platform for hosting and orchestrating your applications that utilize large language models. It allows you to deploy and manage your applications efficiently.

2. Validation and Benchmarking: Pulze.ai offers validation and benchmarking capabilities to ensure the performance and accuracy of your large language models. You can validate your models against various metrics and benchmarks to ensure their quality.

3. LLM Caching: Pulze.ai implements advanced LLM caching techniques to optimize model performance and reduce latency. This is particularly important for applications that require real-time or near real-time responses.

4. LLM Logging and Observability: Pulze.ai provides comprehensive logging and observability features. You can monitor all model requests and performance metrics, gain insights into model performance, and ma